# Variables and Memory

Notes taken from Udacity Cpp Nanodregree

<img width="400" src="figs/virtualmemory.png">

** Figure 1: Virtual Memory **



* The **stack** is a contiguous memory block with a fixed maximum size. If a program exceeds this size, it will crash. The stack is used for storing automatically allocated variables such as local variables or function parameters. If there are multiple threads in a program, then each thread has its own stack memory. New memory on the stack is allocated when the path of execution enters a scope and freed again once the scope is left. It is important to know that the stack is managed "automatically" by the compiler, which means we do not have to concern ourselves with allocation and deallocation.

* The **heap** (also called "free store" in C++) is where data with dynamic storage lives. It is shared among multiple threads in a program, which means that memory management for the heap needs to take concurrency into account. This makes memory allocations in the heap more complicated than stack allocations. In general, managing memory on the heap is more (computationally) expensive for the operating system, which makes it slower than stack memory. Contrary to the stack, the heap is not managed automatically by the system, but by the programmer. If memory is allocated on the heap, it is the programmer’s responsibility to free it again when it is no longer needed. If the programmer manages the heap poorly or not at all, there will be trouble.

* The **BSS** (Block Started by Symbol) segment is used in many compilers and linkers for a segment that contains global and static variables that are initialized with zero values. This memory area is suitable, for example, for arrays that are not initialized with predefined values.

* The **Data segment** serves the same purpose as the BSS segment with the major difference being that variables in the Data segment have been initialized with a value other than zero. Memory for variables in the Data segment (and in BSS) is allocated once when a program is run and persists throughout its lifetime.


### Memory allocation

* **Static memory allocation** is performed for static and global variables, which are stored in the BSS and Data segment. Memory for these types of variables is allocated once when your program is run and persists throughout the life of your program.

* **Automatic memory allocation** is performed for function parameters as well as local variables, which are stored on the stack. Memory for these types of variables is allocated when the path of execution enters a scope and freed again once the scope is left.

* **Dynamic memory allocation** is a possibility for programs to request memory from the operating system at runtime when needed. This is the major difference to automatic and static allocation, where the size of the variable must be known at compile time. Dynamic memory allocation is not performed on the limited stack but on the heap and is thus (almost) only limited by the size of the address space.


# Properties of Stack Memory

* The stack is a **contiguous block of memory**. It will not become fragmented (as opposed to the heap) and it has a fixed maximum size.

* When the **maximum size of the stack** memory is exceeded, a program will crash.

* Allocating and deallocating **memory is fast** on the stack. It only involves moving the stack pointer to a new position.

* When using **multiple threads** (as in concurrent programming), it is important to know that **each thread has its own stack memory** - which can be considered thread-safe. 


### Example 01 Stack Growth and Contraction

In this example we can see that memory is contigouos for i = 1, 2, 4. But for function "MyFunc()"  it uses a part of the memory that is before the main() declaration

```
1: 0x7fff20c3f53c 
2: 0x7fff20c3f540 
3: 0x7fff20c3f514 -> MyFunc()
4: 0x7fff20c3f544 
``` 

Between 2 and 3, the address pointer is moved by 0x28.
MyFunc, the compiler needs to store additional data such as the return address. 


<img width="400" src="figs/growthandcontracts.png">

** Figure 1: Stack Growth and Contracts **

When a thread is created, stack memory is allocated by the operating system as a contiguous block. With each new function call or local variable allocation, the stack pointer is moved until eventually it will reach the bottom of said memory block. Once it exceeds this limit (which is called "stack overflow"), the program will crash. We will try to find out the limit of your computer’s stack memory in the following exercise.

### Pointers vs. References


As we have seen in the examples above, the use of pointers and references to directly manipulate function arguments in a memory-effective way is very similar. Let us compare the two methods in the code on the right.

As can be seen, pointer and reference are both implemented by using a memory address. In the case of AddFour the caller does not even realize that val might be modified while with AddSix, the reference to val has to be explicitly written by using &.

If passing by value needs to be avoided, both pointers and references are a way to achieve this. The following selection of properties contrasts the two methods so it will be easier to decide which to use from the perspective of the use-case at hand:

* Pointers can be declared without initialization. This means we can pass an uninitialized pointer to a function who then internally performs the initialization for us.

* Pointers can be reassigned to another memory block on the heap.

* References are usually easier to use (depending on the expertise level of the programmer). Sometimes however, if a third-party function is used without properly looking at the parameter definition, it might go unnoticed that a value has been modified.



Now, we will compare the three strategies we have seen so far with regard to stack memory usage. Consider the code on the right.

After creating a local variable i in main to give us the address of the stack bottom, we are passing i by-value to our first function. Inside CallByValue, the memory address of a local variable j is printed to the console, which serves as a marker for the stack pointer. With the second function call in main, we are passing a reference to i to CallByPointer. Lastly, the function CallByReference is called in main, which again takes the integer i as an argument. However, from looking at main alone, we can not tell wether i will be passed by value or by reference.

On my machine, when compiled with g++ (Apple clang version 11.0.0), the program produces the following output:

    stack bottom: 0x7ffeefbff698
    call-by-value: 0x7ffeefbff678
    call-by-pointer: 0x7ffeefbff674
    call-by-reference: 0x7ffeefbff674

Depending on your system, the compiler you use and the compiler optimization techniques, you man not always see this result. In some cases

Let us take a look at the respective differences to the stack bottom in turn:

* CallByValue requires 32 bytes of memory. As discussed before, this is reserved for e.g. the function return address and for the local variables within the function (including the copy of i).

* CallByPointer on the other hand requires - perhaps surprisingly - 36 bytes of memory. Let us complete the examination before going into more details on this result.

Let us take a look at the size of the various parameter types using the sizeof command:

    printf("size of int: %lu\n", sizeof(int));
    printf("size of *int: %lu\n", sizeof(int *));

The output here is

    size of int: 4
    size of *int: 8

Obviously, the size of the pointer variable is larger than the actual data type. As my machine has a 64 bit architecture, an address requires 8 byte.

As an experiment, you could use the -m32 compiler flag to build a 32 bit version of the program. This yields the following output:

    size of int: 4
    size of *int: 4

In order to benefit from call-by-reference, the size of the data type 


# Heap

Let us take a look at some properties of heap memory:

* As opposed to local variables on the stack, memory can now be allocated in an arbitrary scope (e.g. inside a function) without it being deleted when the scope is left. Thus, as long as the address to an allocated block of memory is returned by a function, the caller can freely use it.

* Local variables on the stack are allocated at compile-time. Thus, the size of e.g. a string variable might not be appropriate as the length of the string will not be known until the program is executed and the user inputs it. With local variables, a solution would be to allocate a long-enough array of and hope that the actual length does not exceed the buffer size. With dynamically allocated heap memory, variables are allocated at run-time. This means that the size of the above-mentioned string variable can be tailored to the actual length of the user input.

* Heap memory is only constrained by the size of the address space and by the available memory. With modern 64 bit operating systems and large RAM memory and hard disks the programmer commands a vast amount of memory. However, if the programmer forgets to deallocate a block of heap memory, it will remain unused until the program is terminated. This is called a "memory leak".

* Unlike the stack, the heap is shared among multiple threads, which means that memory management for the heap needs to take concurrency into account as several threads might compete for the same memory resource.

* When memory is allocated or deallocated on the stack, the stack pointer is simply shifted upwards or downwards. Due to the sequential structure of stack memory management, stack memory can be managed (by the operating system) easily and securely. With heap memory, allocation and deallocation can occur arbitrarily, depending on the lifetime of the variables. This can result in fragmented memory over time, which is much more difficult and expensive to manage.


### Memory Fragmentation

A classic symptom of memory fragmentation is that you try to allocate a large block and you can’t, even though you appear to have enough memory free. On systems with virtual memory however, this is less of a problem, because large allocations only need to be contiguous in virtual address space, not in physical address space.

    C   : malloc and free
    C++ : new and delete 

## Allocating Dynamic Memory

To reserve memory on the heap, one of the two functions malloc (stands for Memory Allocation) or calloc (stands for Cleared Memory Allocation) is used. The header file stdlib.h or malloc.h must be included to use the functions.

Here is the syntax of malloc and calloc in C/C++:

    pointer_name = (cast-type*) malloc(size);
    pointer_name = (cast-type*) calloc(num_elems, size_elem);

malloc is used to dynamically allocate a single large block of memory

calloc is used to dynamically allocate the specified number of blocks of memory of the specified type. 


* free can only free memory that was reserved with malloc or calloc.

* free can only release memory that has not been released before. Releasing the same block of memory twice will result in an error.

* Memory allocated with malloc or calloc is not subject to the familiar rules of variables in their respective scopes. This means that they exist independently of block limits until they are released again or the program is terminated. However, the pointers which refer to such heap-allocated memory are created on the stack and thus only exist within a limited scope. As soon as the scope is left, the pointer variable will be lost - but not the heap memory it refers to.




## Comparing malloc and new

With the introduction of classes and object oriented programming in C++ however, memory allocation and deallocation has become more complex: When an object is created, its constructor needs to be called to allow for member initialization. Also, on object deletion, the destructor is called to free resources and to allow for programmer-defined clean-up tasks. For this reason, C++ introduces the operators new / delete, which represent the object-oriented counterpart to memory management with malloc / free.

If we were to create a C++ object with malloc, the constructor and destructor of such an object would not be called.

Before we go into further details of new/delete, let us briefly summarize the major differences between malloc/free and new/delete:

* Constructors / Destructors Unlike malloc( sizeof(MyClass) ), the call new MyClass() calls the constructor. Similarly, delete calls the destructor.

* Type safety malloc returns a void pointer, which needs to be cast into the appropriate data type it points to. This is not type safe, as you can freely vary the pointer type without any warnings or errors from the compiler as in the following small example: MyObject *p = (MyObject*)malloc(sizeof(int));

In C++, the call MyObject *p = new MyObject() returns the correct type automatically - it is thus type-safe. 

**Operator Overloading** As malloc and free are functions defined in a library, their behavior can not be changed easily. The new and delete operators however can be overloaded by a class in order to include optional proprietary behavior. We will look at an example of overloading new further down in this section

# New and Delete

### To Call new
1. Memory is allocated to hold a new object of type MyClass
2. A new object of type MyClass is constructed within the allocated memory by calling the constructor of MyClass
### To Call delete
1. The object of type MyClass is destroyed by calling its destructor
2. The memory which the object was palced in is deallocated.

## Optimizing Performance with **Placement new**

In some cases, it makes sense to separate memory allocation from object construction. Consider a case where we need to reconstruct an object several times. If we were to use the standard new/delete construct, memory would be allocated and freed unnecessarily as only the content of the memory block changes but not its size. By separating allocation from construction, we can get a significant performance increase. 

    void *memory = malloc(sizeof(MyClass));
    MyClass *object = new (memory) MyClass;



### Overloading new and delete

    05_overloading_new_delete.cpp
    

### Overloading new[] and delete[]

    05_overload_new_delete_array.cpp
    
Let us consider the example on the right, which has been slightly modified to allocate an array of objects instead of a single one.

Interestingly, the memory requirement is larger than expected: With new, the block size was 4 bytes, which is exactly the space required for a single integer. Thus, with three integers, it should now be 12 bytes instead of 20 bytes. The reason for this is the memory allocation overhead that the compiler needs to keep track of the allocated blocks of memory - which in itself consumes memory. If we change the above call to e.g. new MyClass[100](), we will see that the overhead of 8 bytes does not change: 

    new: Allocating 408 bytes of memory
    Constructor is called
    …
    Destructor is called
    delete: Memory is freed again 



### Reasons for overloading new and delete

Now that we have seen how to overload the new and delete operators, let us summarize the major scenarios where it makes sense to do this:

1. The overloaded new operator function allows to add additional parameters. Therefore, a class can have multiple overloaded new operator functions. This gives the programmer more flexibility in customizing the memory allocation for objects.

2. Overloaded the new and delete operators provides an easy way to integrate a mechanism similar to garbage collection capabilities (such as in Java), as we will shorty see later in this course.

3. By adding exception handling capabilities into new and delete, the code can be made more robust.

4. It is very easy to add customized behavior, such as overwriting deallocated memory with zeros in order to increase the security of critical application data.



## Overview of memory management problems

1. Memory Leaks Memory leaks occur when data is allocated on the heap at runtime, but not properly deallocated. A program that forgets to clear a memory block is said to have a memory leak - this may be a serious problem or not, depending on the circumstances and on the nature of the program. For a program that runs, computes something, and quits immediately, memory leaks are usually not a big concern. Memory leaks are mostly problematic for programs that run for a long time and/or use large data structures. In such a case, memory leaks can gradually fill the heap until allocation requests can no longer be properly met and the program stops responding or crashes completely. We will look at an example further down in this section.

2. Buffer Overruns Buffer overruns occur when memory outside the allocated limits is overwritten and thus corrupted. One of the resulting problems is that this effect may not become immediately visible. When a problem finally does occur, cause and effect are often hard to discern. It is also sometimes possible to inject malicious code into programs in this way, but this shall not be discussed here. 


    char str[5];
    strcpy(str,"BufferOverrun");
    printf("%s",str);

3. Uninitialized Memory Depending on the C++ compiler, data structures are sometimes initialized (most often to zero) and sometimes not. So when allocating memory on the heap without proper initialization, it might sometimes contain garbage that can cause problems.

   Generally, a variable will be automatically initialized in these cases:

* it is a class instance where the default constructor initializes all primitive types
* array initializer syntax is used, such as int a[10] = {}
* it is a global or extern variable
* it is defined static

The behavior of the following code is potentially undefined:

    int a;
    int b=a*42;
    printf("%d",b);

4. Incorrect pairing of allocation and deallocation Freeing a block of memory more than once will cause a program to crash. This can happen when a block of memory is freed that has never been allocated or has been freed before. Such behavior could also occur when improper pairings of allocation and deallocation are used such as using malloc() with delete or new with free().

   In this first example, the wrong new and delete are paired


    double *pDbl=new double[5];
    delete pDbl;

   In this second example, the pairing is correct but a double deletion is performed:

    char *pChr=new char[5];
    delete[] pChr;
    delete[] pChr;


5. Invalid memory access This error occurs then trying to access a block of heap memory that has not yet or has already been deallocated.

   In this example, the heap memory has already been deallocated at the time when strcpy() tries to access it:


    char *pStr=new char[25];
    delete[] pStr;
    strcpy(pStr, "Invalid Access");


## Finding memory leaks

Xcode
Visual Studio
Valgrind -> free. 

        g++ 06_memory_leaks_debugging.cpp

        valgrind --leak-check=full --show-leak-kinds=all --track-origins=yes --log-file=/home/workspace/valgrind-out.txt /home/workspace/a.out

        cat valgrind-out.txt


Finds the leak in the pointer not deleted.
==21928== LEAK SUMMARY:
==21928==    definitely lost: 40 bytes in 1 blocks